# Data Processing

In [ ]:
%pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.1 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

dataset = load_dataset("wmt14",'de-en')

Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

Extracting data files: 0it [00:00, ?it/s]

Generating train split:   0%|          | 0/4508785 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3003 [00:00<?, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 4508785
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 3000
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 3003
    })
})

In [ ]:
train, test, val = dataset['train'], dataset['test'], dataset['validation']

In [ ]:
import pandas as pd

train = pd.DataFrame(train['translation'])
test = pd.DataFrame(test['translation'])
val = pd.DataFrame(val['translation'])


In [ ]:
train.iloc[2]['de'], train.iloc[2]['en']

('Wie Sie feststellen konnten, ist der gefürchtete "Millenium-Bug " nicht eingetreten. Doch sind Bürger einiger unserer Mitgliedstaaten Opfer von schrecklichen Naturkatastrophen geworden.',
 "Although, as you will have seen, the dreaded 'millennium bug' failed to materialise, still the people in a number of countries suffered a series of natural disasters that truly were dreadful.")

In [ ]:
import torch
torch.manual_seed(1)

import nltk

## Character Casing

In [ ]:
"""
Lowercase all words.
"""
def character_casing(text):
    lower_text = text.lower()
    return lower_text

for data in [train, test, val]:
    data.applymap(character_casing)

## AutoTokenizer

In [ ]:
%pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.4 MB/s eta 0:00:00


In [ ]:
from transformers import MarianMTModel, MarianTokenizer, MarianConfig

tokenizer = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-de-en")
model = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-de-en")
config = MarianConfig.from_pretrained("Helsinki-NLP/opus-mt-de-en")


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/797k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/768k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/298M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [ ]:
# get the embedding dimension
embedding_dim = config.hidden_size
embedding_dim

512

In [ ]:
# get the vocabulary size
vocab_size = config.vocab_size
vocab_size

58101

In [ ]:
vocab = tokenizer.get_vocab()

In [ ]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [ ]:
sub = 10000
tokenized = tokenizer(list(train["de"][:sub]), text_target=list(train["en"][:sub]), return_tensors="pt", padding=True, truncation=True, add_special_tokens=True)
torch.save(tokenized, 'train_tokenized_{}.pth'.format(sub))
torch.save(train["en"][:sub], 'references_{}.pth'.format(sub))